## 원하는 페이지까지 크롤링하기

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
from time import sleep
import re

In [2]:
article_dict = {"날짜":[],
               "기사_제목": [],
               "기사_url" : [],
               "기사_내용":[]
               }

# 첫 번째 기사 url 
links = []
first_url = 'https://www.earthmoversmagazine.co.uk/news.htm?news_pg=1'
first_html = requests.get(first_url).text
first_soup = BeautifulSoup(first_html, 'lxml')

first_news = first_soup.find('div', class_='news')
first_link = 'https://www.earthmoversmagazine.co.uk' + first_news.find('a')['href']
links.append(first_link)
article_dict['기사_url'].append(first_link)

for i in tqdm(range(1,22)):
    article_url = 'https://www.earthmoversmagazine.co.uk/news.htm?news_pg={}'.format(i)
    article_html = requests.get(article_url).text
    soup = BeautifulSoup(article_html, 'lxml')
    
    # 날짜, 기사 제목 태그 조회
    remains = soup.find_all('div', class_='col paddedSlim fullWidth')

    for remain in remains:
        # 날짜
        day = " ".join(remain.find_all('span')[0].text.split()[-3:])
        article_dict['날짜'].append(day) 
        # 기사 제목
        title = remain.find('h3').text
        article_dict['기사_제목'].append(title)
    sleep(0.3)
    
    # 두 번째 기사부터~ url 태그 조회
    the_rest = soup.find('div', class_='col_thirds_offset colPadded')
    
    for href in the_rest.find_all('li'):
        if len(href.find('a')['href']) <= 22 : continue
        url = 'https://www.earthmoversmagazine.co.uk' + href.find('a')['href']
        article_dict['기사_url'].append(url)
        # 기사 본문을 추출하기 위한 links 리스트
        links.append(url)
    sleep(0.3) 
    
# 기사 본문
for link in tqdm(links):
    contents_html = requests.get(link).text
    contents_soup = BeautifulSoup(contents_html, 'lxml')

    refine_text = str(contents_soup.find('article', class_='newsview').find_all('p')[1:-1])
    refine_text = re.sub('<br/>|<p>|\n|</p>,|</p>|<b>|</b>|<strong>|</strong>|amp;','', refine_text).strip()
    refine_text = re.sub('\r|\xa0',' ', refine_text).strip()
    # print(refine_text)
    # 위 코드를 출력하면 리스트로 출력된다.
    # 출력된 리스트를 문자열로 바꾸기 위해 아래의 과정을 거친다.
    str_refine_text = refine_text.replace('[', '')
    str_refine_text = str_refine_text.replace(']', '')

    # 같은 p태그로 출력되는 내용 중
    # 'Got a news story?' 이후에 출력되는 부분을 제거하고 본문만 출력한다.
    string = "Got a news story?"
    clear_text = str_refine_text.split(string)
    if len(clear_text) -1 == 1:
        article_dict['기사_내용'].append(clear_text[0])
    else:
        n = len(clear_text) - (len(clear_text)-1)
        article_dict['기사_내용'].append(clear_text[:n])
    sleep(0.3) 
        

100%|████████████████████████████████████████████████████████████████████████████████| 169/169 [04:45<00:00,  1.69s/it]


In [3]:
print('key값 개수:', len(list(article_dict.keys())))
print('value값 개수:', len(list(article_dict.values())))

key값 개수: 4
value값 개수: 4


### 기사_내용에 중복값 확인

In [4]:
print(len(article_dict['날짜']))
print(len(article_dict['기사_제목']))
print(len(article_dict['기사_url']))
print(len(article_dict['기사_내용']))

169
169
169
169


### DataFrame 으로 저장하기

In [5]:
df = pd.DataFrame(article_dict)
df.to_csv("europe_crawling/earthmoversmagazine.csv", index=False)
df.to_excel("europe_crawling/earthmoversmagazine.xlsx", encoding='utf-8', index=False)

In [6]:
df1 = pd.read_csv("europe_crawling/earthmoversmagazine.csv")
display(df1.shape, df1.head())

(169, 4)

,날짜,기사_제목,기사_url,기사_내용
0,22 May 2023,EMS to display Optimas kit at Plantworx,https://www.earthmoversmagazine.co.uk/news/vie...,South Wales-based machinery dealer EMS Limited...
1,19 May 2023,Screencore XJ Dual-Power tracked jaw crusher,https://www.earthmoversmagazine.co.uk/news/vie...,Screencore has launched a new XJ Dual-Power tr...
2,18 May 2023,Engcon DC3 control system for Cat excavators,https://www.earthmoversmagazine.co.uk/news/vie...,Engcon’s next generation DC3 control system is...
3,18 May 2023,HWM Group launches new website,https://www.earthmoversmagazine.co.uk/news/vie...,"Winchester-based company, HWM Group is celebra..."
4,17 May 2023,Sennebogen 728e with Xwatch Safety System,https://www.earthmoversmagazine.co.uk/news/vie...,"Coombes, a provider of forestry and rail servi..."
